In [ ]:
from main import main
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
from matplotlib import rcParams
import time

rcParams['animation.embed_limit'] = 1000  # Größere Animationen einbetten

size = main()  # Ruft die `main()` Funktion auf und speichert die Größe der Welt

# Lade die Simulationsdaten
with open("simulation_data.json", "r") as file:
    data = json.load(file)

# Extrahiere die Zeitschritte und Anzahl der Körper
timesteps = sorted(map(int, data.keys()))
num_bodies = len(data[str(timesteps[0])]["positions"][0])  # Anzahl der Körper

# Konvertiere JSON-Daten in NumPy-Arrays
coordinates = [
    np.array(data[str(t)]["positions"]).T for t in timesteps
]

# Initialisiere das 3D-Plot-Fenster
fig = plt.figure(figsize=(8, 8))
ax_scatter = fig.add_subplot(111, projection="3d")

scatter = ax_scatter.scatter([], [], [], s=50, c="blue", animated=True)

# Setze die Achsenlimits basierend auf `size`
ax_scatter.set_xlim(-size , size)
ax_scatter.set_ylim(-size, size)
ax_scatter.set_zlim(-size, size)

ax_scatter.set_xlabel("X-coordinate")
ax_scatter.set_ylabel("Y-coordinate")
ax_scatter.set_zlabel("Z-coordinate")
ax_scatter.set_title("3D-Simulation Animation")

# Update-Funktion für die Animation
def update(frame):
    """Aktualisiert die Positionen der Körper für jeden Zeitschritt."""
    points = coordinates[frame]
    scatter._offsets3d = (points[:, 0], points[:, 1], points[:, 2])  # X, Y, Z aktualisieren
    ax_scatter.set_title(f"Time Step {frame}")
    return scatter,

print("Creating animation...")
start_time_animation = time.time()

# Erstelle die Animation
ani = FuncAnimation(fig, update, frames=len(timesteps), blit=False, interval=50)

plt.close()  # Verhindert doppelte Anzeige

# Animation in Jupyter anzeigen
display(HTML(ani.to_jshtml()))

end_time_animation = time.time()
print(f"I took {round(end_time_animation - start_time_animation, 2)} seconds to create the animation.")
